# Colouring of graphs of polytopes

Guillermo Pineda-Villavicencio and Julien Ugon
Deakin University (Australia)

## Introdution

We report on the results of experiments on the validity of Hajós’ and Thomson’s conjectures on graphs of polytopes.
This is a Sage notebook, and therefore, to run it you need to have Sage (Stein and Joyner 2005) installed.

In order to run this program, please download the files provided at http://www-imai.is.s.u-tokyo.ac.jp/~hmiyata/oriented_matroids/ and unzip the files in the folder.
The list of files to  download are:
polytope_n5d4.txt
polytope_n6d4.txt
polytope_n7d4.txt
polytope_n8d4.txt
polytope_n6d5.txt
polytope_n7d5.txt
polytope_n8d5.txt
polytope_n9d5.zip *(this file needs to be unziped)*
polytope_n7d6.txt
polytope_n8d6.txt
polytope_n9d6.txt
polytope_n8d7.txt
polytope_n9d7.txt

These files contain all $d$-polytopes on $n$ vertices for the pairs $(d,n)=(4,6-8),(5,7-9),(6,8-9),(7,9)$.

## Initial functions

In [1]:
"""
Get the list of facets from a file:
"""
def getFacets(fname):
    if isinstance(fname,str):
        try:
            return getFacets(open(fname,"r"))
        except:
            return []
    
    return [[Set(eval(f)) for f in l.split()] for l in fname]

In [2]:
def intersect(*args):
    if(len(args)) == 1:
        return args[0]
    return args[0].intersection(intersect(*args[1:]))

def union(*args):
    if(len(args)) == 1:
        return args[0]
    return args[0].union(intersect(*args[1:]))

In [3]:
"""
Given a facet-vertex incidence, give the edges.

This works as follows:
- take all possible pairs of vertices (C(n,2));
- For each pair, take all the facets containing both vertices
- The  intersection of these facets is precisely these two vertices if and only if it's an edge.
"""
def findEdges(facets):
    vertices = union(*facets)
    #take all pairs of vertices:
    V = [(v,w) for v in vertices for w in vertices if v<w]
    L = []
    U = Set(vertices)
    for (v,w) in V:
        F = [f for  f in facets if v in f and w in f]
        E = intersect(*F)
        if E.cardinality() == 2:
            L.append(E)
    return [tuple(e) for e in L]

# Hajós’ conjecture

Hajós’ conjecture states that:

For every $t\geq 2$, every graph with noKt-topological-minor is
$(t−1)$-vertex-colourable.

## Small graphs

We tested Hajós’ conjecture on the graph of all the polytopes provided
by (Miyata, Moriyama, and Fukuda n.d.). Namely, we applied the following
algorithm:

1.  Compute the graph of the polytope;

2.  Compute the chromatic number $t$ of the graph;

3.  Verify that $K_t$ is a topological minor of the graph.



In [4]:
def findLargestCompleteTopologicalMinor(H,n):
    k = n
    while H.topological_minor(graphs.CompleteGraph(k)) :
        k = k+1
    return k

def findLargestCompleteMinor(H,n):
    k = n
    while not (H.minor(graphs.CompleteGraph(k))) :
        k = k-1
    return k

In [5]:
"""
Check Hajos' conjecture on graphs of polytopes. More specifically:
- Compute  the chromatic number t of the graph;
- Check that the graph has K_t as a minor.
"""
def checkHajos(fname):
    fpol = open(fname,"r")
    lines = fpol.readlines()
    fpol.close()
    polys = []
    for l in lines:
       P = Polyhedron(vertices=eval(l[l.index("["):]),base_ring=QQ)
       G = P.graph()
       C = G.chromatic_number();
       #m = findLargestCompleteTopologicalMinor(G,6)
       m = G.minor(graphs.CompleteGraph(C))
       #t = findLargestCompleteMinor(G,10)
       polys.append((P,G,(C,m)))
    return polys

In [6]:
"""
Check Hajos' conjecture on graphs of polytopes. More specifically:
- Compute  the chromatic number t of the graph;
- Check that the graph has K_t as a minor.
"""
def checkHajos2(fname,nvertices):
    polys = []
    for facets in getFacets(fname):
        if facets:
            edges = findEdges(facets)
            G = Graph(edges)
            C = G.chromatic_number();
            #m = findLargestCompleteTopologicalMinor(G,6)
            m = G.minor(graphs.CompleteGraph(C))
            #t = findLargestCompleteMinor(G,10)
            polys.append((facets,G,(C,m)))
    return polys

In [7]:
# Read the polyopes with dimension 4 to 8 and number of vertices 6 to 10:
hajos = [checkHajos2("polytope_n%dd%d.txt"%(n,d),n) for n in range(6,10) for d in range(4,8)]
hajos2 = [h for h in hajos if h != []]

# This will return true if there is a graph that does not satisfy Hajós' conjecture in the list.
any([d[2] == false for dat in hajos2 for d in dat])

False

## Catlin’s Counter examples

In this section we consider Catlin’s famous counter-example to Hajós’
conjecture (Catlin 1979). In this paper, Catlin gives a family of graphs
with chromatic number $n$ that contain no subdivided $K_n$ as a
subgraph. The first counter-example is constructed by removing two non-adjacent vertices $v_1$ and $v_2$ from $L_{_3C_5}$ - namely, Catlin shows that $G = L(_3C_5) - \{v_1,v_2\}$ contradicts Hajós' conjecture. 

Here we show that $G$ is not the graph of a polytope.

In [8]:
# We start by constructing Catlin's counter-example:

lm35=Graph(multiedges=False);
lm35.add_edges([(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(2,3),(2,4),(2,5),(2,6),(2,7),(2,8),(3,4),(3,5),(3,4),(3,9),(3,10),(4,5),(4,9),(4,10),(5,9),(5,10),(6,7),(6,8),(6,11),(6,12),(6,13),(7,8),(7,11),(7,12),(7,13),(8,11),(8,12),(8,13),(9,10),(9,11),(9,12),(9,13),(10,11),(10,12),(10,13),(11,12),(11,13),(12,13)])
lm35.coloring()
lm35.vertex_connectivity()
lm35.topological_minor(graphs.CompleteGraph(6));

First note that $G$ is not planar, and so cannot be the graph
of a 3-polytope. Since it is not 5-connected (the vertices
$\{1,2,9,10\}$ disconnect the graph, it
cannot be the graph of a $d$-polytope for $d\geq 5$. So, we only need to study the
case $d=4$.

The graphs of the facets of any 4-polytope must be planar, 3-connnected,
and should not disconnect the entire graph.

In [9]:
# Get all the connected subgraphs of lm35 of order at most 10.
LGraphs =list(lm35.connected_subgraph_iterator(k=10))

P = set([1,2,9,10])

"""
A filter function that  accepts only graphs that are:
- at least 4 vertices
- minimum degree 3
- is planar
- the connectivity is at least 3
"""
def validSubgraph(H):
    l = lm35.copy()
    l.delete_vertices(H.vertices())
    return  H.num_verts() >= 4 and  min(H.degree()) > 2  and H.is_planar() and  H.vertex_connectivity() >2 and l.is_connected()
    # and not set(H.vertices()).issubset(P)

#l2  =filter(validSubgraph,lm35.connected_subgraph_iterator(k=10))
l2  = list(filter(validSubgraph,LGraphs))

Let us consider those that
contain Vertex $3$ and either of vertices 1,2 and 9. The graphs of
these facets must be one of the following subgraphs of $G$.

We start by finding the subsets containing the edge {1,3}.

In [10]:
# Get the 3-connected, planar subgraphs containing the edge (1,3):
S13 = [l for l in l2 if (1,3,None) in l.edges() or (3,1,None) in l.edges()]
print([s.vertices() for s  in S13])

[[1, 3, 4, 5, 10], [1, 2, 3, 4], [1, 2, 3, 5], [1, 3, 4, 5], [1, 3, 4, 5, 9]]


The edge $\{1,3\}$ must belong to at least three facets amongs the candidates:
$\{1,2,3,4\}$,$\{1,2,3,5\}$,$\{1,3,4,5,9\}$,$\{1,3,4,5,10\}$,
and $\{1,3,4,5\}$. At least one of these three facets must contain the
vertices $\{3,4,5\}$. 


By a similar argument, the edge $\{2,3\}$ must
also belong to a facet containing the vertices $\{3,4,5\}$:

In [11]:
S23 = [l for l in l2 if (2,3,None) in l.edges() or (3,2,None) in l.edges()]
print([s.vertices() for s  in S23])

[[2, 3, 4, 5, 10], [1, 2, 3, 4], [1, 2, 3, 5], [2, 3, 4, 5], [2, 3, 4, 5, 9]]


 The
intersection between these two facets is either the triangle
$\{3,4,5\}$, or contains the four vertices $\{3,4,5,k\}$ for
$k\in \{9,10\}$. We can rule out the latter case, since this would
require the graph of this 2-face to be complete.

We can conclude that the sets of vertices $\{1,3,4,5\}$ and
$\{2,3,4,5\}$ both form a facet of the polytope.

The edge$\{3,9\}$ must also belong to at least three facets, one of
which contains the ridge $\{3,4,5\}$:

In [12]:
S39 = [l for l in l2 if (3,9,None) in l.edges() or (9,3,None) in l.edges()]
print([s.vertices() for s  in S39])

[[3, 4, 9, 10], [3, 5, 9, 10], [1, 3, 4, 5, 9], [2, 3, 4, 5, 9], [3, 4, 5, 9]]


This cannot happen, since this
would imply that the ridge is contained in three distinct facets.

Therefore we conclude that Catlin's example is not the graph of a polytope.

# Thompson’s conjecture

Abigail Thompson made the following conjecture (Kalai 2014):

Graphs of simple $d$-polytopes with an even number of vertices are
$d$-colourable.

We tested these conjecture of small polytopes provided by (Miyata,
Moriyama, and Fukuda n.d.), using the following algorithm. Namely, we
selected all simplicial polytopes in this collection with an even number
of facets. We then verified that the chromatic numbers of each of these
polytopes is at most $d$.

The code is available in (Ugon, n.d.)

In [13]:
"""
This function returns the facets of the dual polytope, given a list of facets for the primal.
"""
def dualFacets(f):
    nvertices = len(f)
    facets = [Set([i for (i,facet) in enumerate(f) if v in facet]) for v in union(*f)]
    return facets

In [14]:
"""
Check Thomson's conjecture on the dual of a d-polytope:
"""
def checkThompson(fname,d):
    polytopes = getFacets(fname)
    simplicialFacets = [facets for facets in polytopes if all(f.cardinality() == d for f in facets)]
    evensimplificialFacets = [facets for facets in simplicialFacets if len(facets)%2 == 0]
    edgeslists = [findEdges(facets) for facets in evensimplificialFacets]
    graphs = [Graph(edges) for edges in edgeslists]
    return [(d,graph_coloring.edge_coloring(G,value_only=true)) for G in graphs]

In [15]:
# Read the polyopes with dimension 4 to 8 and number of vertices 6 to 10:
thompson = [checkThompson("polytope_n%dd%d.txt"%(n,d),d) for n in range(6,10) for d in range(4,8)]

#This will return true if there is a polytope that does not satisfy Thompson's conjecture
any([x[0]<x[1] for x in sum(thompson,[])])

False

# References

<div id="refs" class="references hanging-indent">

<div id="ref-catlin1979hajos">

Catlin, Paul A. 1979. “Hajós’ Graph-Coloring Conjecture: Variations and
Counterexamples.” *Journal of Combinatorial Theory, Series B* 26 (2):
268–74.

</div>

<div id="ref-kal14">

Kalai, Gil. 2014. “Coloring Simple Polytopes and Triangulations.” 2014.
<https://gilkalai.wordpress.com/2014/12/06/coloring-simple-polytopes-and-triangulations/>.

</div>

<div id="ref-matroids">

Miyata, Hiroyuki, Sonoko Moriyama, and Komei Fukuda. n.d.
“Classification of Oriented Matroids.” Accessed January 3, 2020.
<http://www-imai.is.s.u-tokyo.ac.jp/~hmiyata/oriented_matroids/>.

</div>

<div id="ref-SteinJoyner2005">

Stein, William, and David Joyner. 2005. “SAGE: System for Algebra and
Geometry Experimentation.” *ACM SIGSAM Bulletin* 39 (2): 61–64.
<http://www.sagemath.org/files/sage_stein2005.pdf>.

</div>

<div id="ref-repo">

Ugon, Julien. n.d. “Github Repository.”
<https://github.com/ugonj/Graphs-of-Polytopes>.

</div>

